<a href="https://colab.research.google.com/github/ElijahSagaranCSUN/Kaggle-House-Pricing-Project/blob/main/final_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("SignifReg")
install.packages("missForest")
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘car’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rngtools’, ‘randomForest’, ‘foreach’, ‘itertools’, ‘iterators’, ‘doRNG’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [ ]:


library(SignifReg)
library(missForest)
library(car)
library(readr)
library(caret)

train_data <- read.csv("train.csv")
test_data <- read.csv("test.csv")

train_data <- train_data[, -1]

ID <- test_data$ID
test_data <- test_data[, -1]

na_means_none <- c("GarageYrBlt", "Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "GarageType", "GarageFinish", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature")

# GarageYrBlt_train <- train_data$GarageYrBlt
# GarageYrBlt_test <- test_data$GarageYrBlt

# train_data <- train_data[, !names(train_data) %in% "GarageYrBlt"]
# test_data <- test_data[, !names(test_data) %in% "GarageYrBlt"]

sum(is.na(train_data))
sum(is.na(test_data))

for(i in 1:length(na_means_none)){
  train_data[is.na(train_data[, na_means_none[i]]), na_means_none[i]] <- "None"
  test_data[is.na(test_data[, na_means_none[i]]), na_means_none[i]] <- "None"
}

sum(is.na(train_data))
sum(is.na(test_data))

GarageYrBlt_train <- train_data$GarageYrBlt
GarageYrBlt_test <- test_data$GarageYrBlt

train_data <- train_data[, !names(train_data) %in% "GarageYrBlt"]
test_data <- test_data[, !names(test_data) %in% "GarageYrBlt"]

sum(is.na(train_data))
sum(is.na(test_data))

char_cols_train <- sapply(train_data, is.character)
char_cols_test <- sapply(test_data, is.character)

train_data[char_cols_train] <- lapply(train_data[char_cols_train], as.factor)
test_data[char_cols_test] <- lapply(test_data[char_cols_test], as.factor)

imputed_data_train <- missForest(train_data)
imputed_data_test <- missForest(test_data)

complete_train_data <- imputed_data_train$ximp
complete_test_data <- imputed_data_test$ximp

sum(is.na(complete_train_data))
sum(is.na(complete_test_data))

complete_train_data <- cbind(complete_train_data, GarageYrBlt_train)
colnames(complete_train_data)[colnames(complete_train_data) == "GarageYrBlt_train"] <- "GarageYrBlt"

complete_test_data <- cbind(complete_test_data, GarageYrBlt_test)
colnames(complete_test_data)[colnames(complete_test_data) == "GarageYrBlt_test"] <- "GarageYrBlt"

full_model <- lm(SalePrice ~ ., data = complete_train_data)

set_1 <- c("X1stFlrSF", "X2ndFlrSF", "LowQualFinSF", "GrLivArea")
set_2 <- c("BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF")

train_data_for_set_1 <- complete_train_data[, set_1]
train_data_for_set_2 <- complete_train_data[, set_2]

test_data_for_set_1 <- complete_test_data[, set_1]
test_data_for_set_2 <- complete_test_data[, set_2]

complete_train_data <- complete_train_data[, !names(complete_train_data) %in% set_1]
complete_train_data <- complete_train_data[, !names(complete_train_data) %in% set_2]

complete_test_data <- complete_test_data[, !names(complete_test_data) %in% set_1]
complete_test_data <- complete_test_data[, !names(complete_test_data) %in% set_2]

pca_results_train_1 <- prcomp(train_data_for_set_1, center = FALSE, scale = FALSE)
pca_results_train_2 <- prcomp(train_data_for_set_2, center = FALSE, scale = FALSE)

pca_results_test_1 <- prcomp(test_data_for_set_1, center = FALSE, scale = FALSE)
pca_results_test_2 <- prcomp(test_data_for_set_2, center = FALSE, scale = FALSE)

pca_scores_train_1 <- pca_results_train_1$x
pca_scores_train_1 <- pca_scores_train_1[, 1]

pca_scores_test_1 <- pca_results_test_1$x
pca_scores_test_1 <- pca_scores_test_1[, 1]

pca_scores_train_2 <- pca_results_train_2$x
pca_scores_train_2 <- pca_scores_train_2[, 1:2]

pca_scores_test_2 <- pca_results_test_2$x
pca_scores_test_2 <- pca_scores_test_2[, 1:2]

complete_train_data <- cbind(complete_train_data, pca_scores_train_1)
complete_train_data <- cbind(complete_train_data, pca_scores_train_2)

complete_test_data <- cbind(complete_test_data, pca_scores_test_1)
complete_test_data <- cbind(complete_test_data, pca_scores_test_2)

colnames(complete_train_data)[colnames(complete_train_data) == "PC2"] <-"PC3"
colnames(complete_train_data)[colnames(complete_train_data) == "PC1"] <-"PC2"
colnames(complete_train_data)[colnames(complete_train_data) == "pca_scores_train_1"] <-"PC1"

colnames(complete_test_data)[colnames(complete_test_data) == "PC2"] <-"PC3"
colnames(complete_test_data)[colnames(complete_test_data) == "PC1"] <-"PC2"
colnames(complete_test_data)[colnames(complete_test_data) == "pca_scores_test_1"] <-"PC1"

full_model <- lm(SalePrice ~ ., data = complete_train_data)

complete_train_data <- complete_train_data[, !names(complete_train_data) %in% c("Exterior2nd", "BsmtCond", "BsmtFinType1")]
complete_test_data <- complete_test_data[, !names(complete_test_data) %in% c("Exterior2nd", "BsmtCond", "BsmtFinType1")]

full_model <- lm(SalePrice ~ ., data = complete_train_data)
null_model <- lm(SalePrice ~ 1, data = complete_train_data)

scope = list(lower = null_model, upper = full_model)
forward_model <- SignifReg(null_model, scope = scope, alpha = 0.05, direction = "forward", criterion = "BIC", adjust.method = "fdr", trace = TRUE)

model_formula <- formula(forward_model)
predictor_names <- c("RoofMatl", "OverallQual", "PC1", "Neighborhood", "BsmtQual", "PC3", "MSSubClass", "KitchenQual", "BsmtExposure", "OverallCond", "Condition2", "YearBuilt", "LotArea", "GarageArea", "PoolArea", "SaleCondition", "ExterQual", "MasVnrArea", "PoolQC", "ScreenPorch", "Street")
final_train_data <- complete_train_data[, c("SalePrice")]
final_predictors <- complete_train_data[, names(complete_train_data) %in% predictor_names]
final_train_data <- cbind(final_train_data, final_predictors)
colnames(final_train_data)[colnames(final_train_data) == "final_train_data"] <- "SalePrice"

model <- lm(sqrt(SalePrice) ~ ., data = final_train_data)
model_formula <- update(model_formula, log(.) ~ .)

set.seed(123)

mape <- function(data, lev = NULL, model = NULL){
  difference <- data$obs - data$pred
  percentage_error <- abs(difference) / data$obs * 100

  mean_absolute_percentage_error <- mean(percentage_error, na.rm = TRUE)

  return(c(MAPE = mean_absolute_percentage_error))
}

combinedSummary <- function(data, lev = NULL, model = NULL){
  default_metrics <- defaultSummary(data, lev, model)

  mape_metric <- mape(data, lev, model)

  return(c(default_metrics, mape_metric))
}

control <- trainControl(method = "cv", numbe = 5, summaryFunction = combinedSummary)

model <- train(
  model_formula,
  data = final_train_data,
  method = "lm",
  trControl = control,
  metric = "MAPE"
)

print(model)

final_model <- lm(model_formula, data = final_train_data)

predictors <- attr(terms(final_model), "term.labels")

predictions <- predict(final_model, newdata = complete_test_data)

final_predictions <- exp(predictions)

submission <- data.frame(SalePrice = final_predictions)
num_rows <- length(submission)

id_vector <- 1461:(1461 + num_rows - 1)
submission$ID <- id_vector

submission <- submission[, c("ID", "SalePrice")]

1        2        3        4        5        6        7        8 
11.82995 11.97737 12.07614 12.16193 12.24283 12.03280 12.12912 11.98700 
       9       10       11       12       13       14       15       16 
12.22451 11.68675 12.10791 11.50227 11.49521 11.93212 11.76909 12.79109 
      17       18       19       20       21       22       23       24 
12.45832 12.59878 12.56698 13.14710 12.66958 12.23508 12.08640 12.00566 
      25       26       27       28       29       30       31       32 
12.11348 12.17541 12.74391 12.36863 12.21107 12.38701 12.13651 11.56620 
      33       34       35       36       37       38       39       40 
12.02469 12.57424 12.56728 12.32341 12.15159 12.02749 12.02166 11.97286 
      41       42       43       44       45       46       47       48 
12.04734 11.97193 12.50391 12.35518 12.29684 12.13109 12.41046 12.17598 
      49       50       51       52       53       54       55       56 
11.98400 11.83645 11.87984 12.10137 11.89180 11.90322 12.26638 11.94029 
      57       58       59       60       61       62       63       64 
12.06096 11.85355 12.25738 11.77121 11.79847 12.05221 11.62350 11.75165 
      65       66       67       68       69       70       71       72 
11.72981 11.52526 11.63617 11.88857 11.84832 12.18268 11.48408 11.52689 
      73       74       75       76       77       78       79       80 
11.94349 11.68959 11.89094 11.59191 11.08849 11.84255 12.18937 11.45291 
      81       82       83       84       85       86       87       88 
11.80730 11.87642 12.11713 11.52457 11.67872 11.81497 11.85536 11.88169 
      89       90       91       92       93       94       95       96 
11.72829 11.76028 11.62359 11.67317 11.79317 11.67616 12.06261 11.52295 
      97       98       99      100      101      102      103      104 
11.51358 11.60995 11.32925 11.81727 11.58581 11.81085 11.66993 11.97201 
     105      106      107      108      109      110      111      112 
11.81150 12.42763 11.36048 12.27878 11.61366 11.85437 11.68300 11.84439 
     113      114      115      116      117      118      119      120 
12.37006 11.67155 12.36085 12.35020 12.10731 11.81290 11.89582 12.13708 
     121      122      123      124      125      126      127      128 
11.95350 11.79504 12.59973 12.27054 11.88352 11.22003 11.55122 11.91792 
     129      130      131      132      133      134      135      136 
11.47132 11.81900 11.45493 11.69374 11.75086 11.77778 11.53899 12.26690 
     137      138      139      140      141      142      143      144 
12.17824 12.33948 12.08727 11.96606 11.16841 11.65017 11.10020 12.61709 
     145      146      147      148      149      150      151      152 
12.31884 12.01205 12.14204 12.17138 12.13450 11.93343 11.79014 12.04472 
     153      154      155      156      157      158      159      160 
11.93999 11.66672 11.40689 11.24843 11.42901 11.75207 11.87668 11.95925 
     161      162      163      164      165      166      167      168 
11.84544 11.80165 12.45023 12.18754 11.73962 12.01002 12.12694 12.57431 
     169      170      171      172      173      174      175      176 
12.13577 12.88034 12.27975 12.41823 12.05088 12.09602 12.06570 11.97843 
     177      178      179      180      181      182      183      184 
12.17882 12.13716 12.09887 12.51552 12.11790 12.40577 12.32410 12.33831 
     185      186      187      188      189      190      191      192 
12.32197 11.94613 11.96285 11.76088 11.85371 11.63281 11.73598 11.51680 
     193      194      195      196      197      198      199      200 
11.56680 11.87994 11.88658 11.86996 11.88707 11.86638 11.81030 11.93710 
     201      202      203      204      205      206      207      208 
12.99648 12.97180 12.86018 13.42786 12.62765 12.74203 12.78083 12.75126 
     209      210      211      212      213      214      215      216 
12.63484 12.76276 12.47104 13.04907 12.54781 12.42176 12.17709 12.18506 
     217      218      219      220      221      222      223   

In [ ]:
library(SignifReg)
library(missForest)
library(car)
library(readr)
library(caret)

In [ ]:
train_data <- read.csv("train.csv")
test_data <- read.csv("test.csv")

train_data <- train_data[, -1]

ID <- test_data$ID
test_data <- test_data[, -1]

na_means_none <- c("GarageYrBlt", "Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "GarageType", "GarageFinish", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature")

# GarageYrBlt_train <- train_data$GarageYrBlt
# GarageYrBlt_test <- test_data$GarageYrBlt

# train_data <- train_data[, !names(train_data) %in% "GarageYrBlt"]
# test_data <- test_data[, !names(test_data) %in% "GarageYrBlt"]

sum(is.na(train_data))
sum(is.na(test_data))

In [ ]:
for(i in 1:length(na_means_none)){
  train_data[is.na(train_data[, na_means_none[i]]), na_means_none[i]] <- "None"
  test_data[is.na(test_data[, na_means_none[i]]), na_means_none[i]] <- "None"
}

sum(is.na(train_data))
sum(is.na(test_data))

[1] 966

[1] 1010

In [ ]:
GarageYrBlt_train <- train_data$GarageYrBlt
GarageYrBlt_test <- test_data$GarageYrBlt

train_data <- train_data[, !names(train_data) %in% "GarageYrBlt"]
test_data <- test_data[, !names(test_data) %in% "GarageYrBlt"]

sum(is.na(train_data))
sum(is.na(test_data))

[1] 966

[1] 1010

In [ ]:
char_cols_train <- sapply(train_data, is.character)
char_cols_test <- sapply(test_data, is.character)

train_data[char_cols_train] <- lapply(train_data[char_cols_train], as.factor)
test_data[char_cols_test] <- lapply(test_data[char_cols_test], as.factor)

imputed_data_train <- missForest(train_data)
imputed_data_test <- missForest(test_data)

complete_train_data <- imputed_data_train$ximp
complete_test_data <- imputed_data_test$ximp

sum(is.na(complete_train_data))
sum(is.na(complete_test_data))

In [ ]:
colnames(complete_train_data)

In [ ]:
colnames(complete_test_data)

In [ ]:
complete_train_data <- cbind(complete_train_data, GarageYrBlt_train)
colnames(complete_train_data)[colnames(complete_train_data) == "GarageYrBlt_train"] <- "GarageYrBlt"

complete_test_data <- cbind(complete_test_data, GarageYrBlt_test)
colnames(complete_test_data)[colnames(complete_test_data) == "GarageYrBlt_test"] <- "GarageYrBlt"

In [ ]:
full_model <- lm(SalePrice ~ ., data = complete_train_data)

In [ ]:
set_1 <- c("X1stFlrSF", "X2ndFlrSF", "LowQualFinSF", "GrLivArea")
set_2 <- c("BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF")

train_data_for_set_1 <- complete_train_data[, set_1]
train_data_for_set_2 <- complete_train_data[, set_2]

test_data_for_set_1 <- complete_test_data[, set_1]
test_data_for_set_2 <- complete_test_data[, set_2]

complete_train_data <- complete_train_data[, !names(complete_train_data) %in% set_1]
complete_train_data <- complete_train_data[, !names(complete_train_data) %in% set_2]

complete_test_data <- complete_test_data[, !names(complete_test_data) %in% set_1]
complete_test_data <- complete_test_data[, !names(complete_test_data) %in% set_2]

pca_results_train_1 <- prcomp(train_data_for_set_1, center = FALSE, scale = FALSE)
pca_results_train_2 <- prcomp(train_data_for_set_2, center = FALSE, scale = FALSE)

pca_results_test_1 <- prcomp(test_data_for_set_1, center = FALSE, scale = FALSE)
pca_results_test_2 <- prcomp(test_data_for_set_2, center = FALSE, scale = FALSE)

pca_scores_train_1 <- pca_results_train_1$x
pca_scores_train_1 <- pca_scores_train_1[, 1]

pca_scores_test_1 <- pca_results_test_1$x
pca_scores_test_1 <- pca_scores_test_1[, 1]

pca_scores_train_2 <- pca_results_train_2$x
pca_scores_train_2 <- pca_scores_train_2[, 1:2]

pca_scores_test_2 <- pca_results_test_2$x
pca_scores_test_2 <- pca_scores_test_2[, 1:2]

complete_train_data <- cbind(complete_train_data, pca_scores_train_1)
complete_train_data <- cbind(complete_train_data, pca_scores_train_2)

complete_test_data <- cbind(complete_test_data, pca_scores_test_1)
complete_test_data <- cbind(complete_test_data, pca_scores_test_2)

In [ ]:
colnames(complete_train_data)[colnames(complete_train_data) == "PC2"] <-"PC3"
colnames(complete_train_data)[colnames(complete_train_data) == "PC1"] <-"PC2"
colnames(complete_train_data)[colnames(complete_train_data) == "pca_scores_train_1"] <-"PC1"

In [ ]:
colnames(complete_test_data)[colnames(complete_test_data) == "PC2"] <-"PC3"
colnames(complete_test_data)[colnames(complete_test_data) == "PC1"] <-"PC2"
colnames(complete_test_data)[colnames(complete_test_data) == "pca_scores_test_1"] <-"PC1"

In [ ]:
full_model <- lm(SalePrice ~ ., data = complete_train_data)

In [ ]:
complete_train_data <- complete_train_data[, !names(complete_train_data) %in% c("Exterior2nd", "BsmtCond", "BsmtFinType1")]
complete_test_data <- complete_test_data[, !names(complete_test_data) %in% c("Exterior2nd", "BsmtCond", "BsmtFinType1")]

full_model <- lm(SalePrice ~ ., data = complete_train_data)
null_model <- lm(SalePrice ~ 1, data = complete_train_data)

scope = list(lower = null_model, upper = full_model)
forward_model <- SignifReg(null_model, scope = scope, alpha = 0.05, direction = "forward", criterion = "BIC", adjust.method = "fdr", trace = TRUE)

In [ ]:
model_formula <- formula(forward_model)
predictor_names <- c("RoofMatl", "OverallQual", "PC1", "Neighborhood", "BsmtQual", "PC3", "MSSubClass", "KitchenQual", "BsmtExposure", "OverallCond", "Condition2", "YearBuilt", "LotArea", "GarageArea", "PoolArea", "SaleCondition", "ExterQual", "MasVnrArea", "PoolQC", "ScreenPorch", "Street")
final_train_data <- complete_train_data[, c("SalePrice")]
final_predictors <- complete_train_data[, names(complete_train_data) %in% predictor_names]
final_train_data <- cbind(final_train_data, final_predictors)
colnames(final_train_data)[colnames(final_train_data) == "final_train_data"] <- "SalePrice"

In [ ]:
model <- lm(sqrt(SalePrice) ~ ., data = final_train_data)
model_formula <- update(model_formula, log(.) ~ .)

In [ ]:
set.seed(123)

mape <- function(data, lev = NULL, model = NULL){
  difference <- data$obs - data$pred
  percentage_error <- abs(difference) / data$obs * 100

  mean_absolute_percentage_error <- mean(percentage_error, na.rm = TRUE)

  return(c(MAPE = mean_absolute_percentage_error))
}

combinedSummary <- function(data, lev = NULL, model = NULL){
  default_metrics <- defaultSummary(data, lev, model)

  mape_metric <- mape(data, lev, model)

  return(c(default_metrics, mape_metric))
}

control <- trainControl(method = "cv", numbe = 5, summaryFunction = combinedSummary)

model <- train(
  model_formula,
  data = final_train_data,
  method = "lm",
  trControl = control,
  metric = "MAPE"
)

In [ ]:
print(model)

In [ ]:
final_model <- lm(model_formula, data = final_train_data)

In [ ]:
summary(final_model)

In [ ]:
predictors <- attr(terms(final_model), "term.labels")
predictors

In [ ]:
predictions <- predict(final_model, newdata = complete_test_data)

In [ ]:
predictions
final_predictions <- exp(predictions)

In [ ]:
final_predictions

In [ ]:
submission <- data.frame(SalePrice = final_predictions)
num_rows <- length(submission)

id_vector <- 1461:(1461 + num_rows - 1)
submission$ID <- id_vector

submission <- submission[, c("ID", "SalePrice")]


In [ ]:
write.csv(submission, "submission.csv", row.names = FALSE)